In [1]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]
vectorstore = Chroma.from_documents(docs, embedding=embeddings)

/Users/bwubete/opt/anaconda3/envs/bpa_ai_sdk_new/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/bwubete/opt/anaconda3/envs/bpa_ai_sdk_new/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI
from llm_model import CienaGPTChatModel

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = CienaGPTChatModel(model_name='test-model', model_id = '5', base_address='https://gptapi.ciena.com/', okta_domain='ciena.okta.com', okta_custom_scope='api', client_secret='uNU1c9Vl01WfkBdQDUivZr0O1pR3qxJpe5js3d1PJhwqGO63WEvqvOXgBux9dRyi', client_id='0oa1x6yhddqyoeoTY0h8')
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

In [4]:
# This example specifies a query and composite filter
retriever.invoke(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

success: 200
{'result': {'id': 0, 'oktaUserId': None, 'openAiId': 'chatcmpl-ADL9NLcQEUnjJOwmDnpYvHSpIAcWz', 'model': 'gpt-4o-2024-05-13', 'title': None, 'choices': [{'message': {'content': '```json\n{\n    "query": "toys",\n    "filter": "and(gt(\\"year\\", 1990), lt(\\"year\\", 2005), eq(\\"genre\\", \\"animated\\"))"\n}\n```', 'role': 'assistant', 'created': '0001-01-01T00:00:00'}, 'index': None, 'finishReason': 'Stop'}], 'toolCalls': [], 'completionUsage': {'id': 0, 'completionTokens': 45, 'promptTokens': 1042, 'totalTokens': 1087}, 'created': '0001-01-01T00:00:00'}, 'isSuccess': True, 'errorMessage': '', 'isNewConversation': True}


[Document(metadata={'genre': 'animated', 'year': 1995}, page_content='Toys come alive and have a blast doing so')]

In [5]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
# prompt = ChatPromptTemplate.from_messages([
#     SystemMessagePromptTemplate.from_message("""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. Context: {context} """),
#     HumanMessagePromptTemplate.from_message("{question}")
# ])
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
# llm = CienaGPTChatModel(model_name='test-model', model_id = '5', base_address='https://gptapi.ciena.com/', okta_domain='ciena.okta.com', okta_custom_scope='api', client_secret='uNU1c9Vl01WfkBdQDUivZr0O1pR3qxJpe5js3d1PJhwqGO63WEvqvOXgBux9dRyi', client_id='0oa1x6yhddqyoeoTY0h8')
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

In [9]:
chain.invoke({'input': "Is there a movie after 1990 but before 2005 that's all about dinosaurs? If not, give me more information about such a movie"})

success: 200
{'result': {'id': 0, 'oktaUserId': None, 'openAiId': 'chatcmpl-ADLLLTN8Hz63uCZhxvZcd6U3j1W8w', 'model': 'gpt-4o-2024-05-13', 'title': None, 'choices': [{'message': {'content': '```json\n{\n    "query": "dinosaurs",\n    "filter": "and(gt(\\"year\\", 1990), lt(\\"year\\", 2005))"\n}\n```', 'role': 'assistant', 'created': '0001-01-01T00:00:00'}, 'index': None, 'finishReason': 'Stop'}], 'toolCalls': [], 'completionUsage': {'id': 0, 'completionTokens': 37, 'promptTokens': 1050, 'totalTokens': 1087}, 'created': '0001-01-01T00:00:00'}, 'isSuccess': True, 'errorMessage': '', 'isNewConversation': True}
success: 200
{'result': {'id': 0, 'oktaUserId': None, 'openAiId': 'chatcmpl-ADLLQOb70d40TDv4LyTg9TUonLXOI', 'model': 'gpt-4o-2024-05-13', 'title': None, 'choices': [{'message': {'content': 'Yes, "Jurassic Park," released in 1993, is a movie about dinosaurs where scientists bring them back to life, leading to chaos. Directed by Steven Spielberg, it is based on Michael Crichton\'s nov

{'input': "Is there a movie after 1990 but before 2005 that's all about dinosaurs? If not, give me more information about such a movie",
 'context': [Document(metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose'),
  Document(metadata={'genre': 'animated', 'year': 1995}, page_content='Toys come alive and have a blast doing so')],
 'answer': 'Yes, "Jurassic Park," released in 1993, is a movie about dinosaurs where scientists bring them back to life, leading to chaos. Directed by Steven Spielberg, it is based on Michael Crichton\'s novel and became a significant cultural phenomenon.'}